In [36]:
import pandas as pd
!pip install nltk

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 25.8 kB/s eta 0:00:00m eta 0:00:010:00:02
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.1/77.1 kB 24.8 kB/s eta 0:00:0031m23.4 kB/s eta 0:00:01

[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [45]:
import nltk
import re
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/manmohan/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [83]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /home/manmohan/nltk_data...


True

In [23]:
data = pd.read_csv("/home/manmohan/Downloads/fakesdata.csv")

In [24]:
data.head()

,unit_id,article_title,article_content,source,date,location,labels
0,1914947530,Syria attack symptoms consistent with nerve ag...,Wed 05 Apr 2017 Syria attack symptoms consiste...,nna,4/5/2017,idlib,0
1,1914947532,Homs governor says U.S. attack caused deaths b...,Fri 07 Apr 2017 at 0914 Homs governor says U.S...,nna,4/7/2017,homs,0
2,1914947533,Death toll from Aleppo bomb attack at least 112,Sun 16 Apr 2017 Death toll from Aleppo bomb at...,nna,4/16/2017,aleppo,0
3,1914947534,Aleppo bomb blast kills six Syrian state TV,Wed 19 Apr 2017 Aleppo bomb blast kills six Sy...,nna,4/19/2017,aleppo,0
4,1914947535,29 Syria Rebels Dead in Fighting for Key Alepp...,Sun 10 Jul 2016 29 Syria Rebels Dead in Fighti...,nna,7/10/2016,aleppo,0


In [25]:
data = data.drop(["source", "date", "location"], axis = 1)

In [26]:
data.head()

,unit_id,article_title,article_content,labels
0,1914947530,Syria attack symptoms consistent with nerve ag...,Wed 05 Apr 2017 Syria attack symptoms consiste...,0
1,1914947532,Homs governor says U.S. attack caused deaths b...,Fri 07 Apr 2017 at 0914 Homs governor says U.S...,0
2,1914947533,Death toll from Aleppo bomb attack at least 112,Sun 16 Apr 2017 Death toll from Aleppo bomb at...,0
3,1914947534,Aleppo bomb blast kills six Syrian state TV,Wed 19 Apr 2017 Aleppo bomb blast kills six Sy...,0
4,1914947535,29 Syria Rebels Dead in Fighting for Key Alepp...,Sun 10 Jul 2016 29 Syria Rebels Dead in Fighti...,0


In [27]:
data.shape

(804, 4)

In [28]:
y = data["labels"]
data = data.drop("labels", axis = 1)

In [29]:
data = data.dropna()

In [30]:
data.shape

(804, 3)

In [31]:
y = y.dropna()

In [32]:
y.shape

(804,)

In [33]:
messages = data.copy()

In [34]:
messages["article_title"][5]

'Suicide bombing kills at least 16 in northeast Syria'

In [84]:
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
lm = WordNetLemmatizer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['article_title'][i])
    review = review.lower()
    review = review.split()
    review = [lm.lemmatize(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [85]:
corpus[0]

'syria attack symptom consistent nerve agent use'

In [86]:
import sklearn

In [103]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 1000, ngram_range = (1,3))
x = cv.fit_transform(corpus).toarray()

In [104]:
x.shape

(804, 1000)

In [105]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size = 0.3, random_state = 2)

In [106]:
cv.get_feature_names_out()[:20]

array(['action', 'activist', 'advance', 'agency', 'ahram', 'ahram online',
       'ahrar', 'ahrar al', 'ahrar al sham', 'aid', 'aid convoy', 'air',
       'air force', 'air raid', 'air raid kill', 'air strike',
       'air strike kill', 'air strike syria', 'airstrike',
       'airstrike kill'], dtype=object)

In [107]:
cv.get_params()

{'analyzer': 'word',
 'binary': False,
 'decode_error': 'strict',
 'dtype': numpy.int64,
 'encoding': 'utf-8',
 'input': 'content',
 'lowercase': True,
 'max_df': 1.0,
 'max_features': 1000,
 'min_df': 1,
 'ngram_range': (1, 3),
 'preprocessor': None,
 'stop_words': None,
 'strip_accents': None,
 'token_pattern': '(?u)\\b\\w\\w+\\b',
 'tokenizer': None,
 'vocabulary': None}

In [108]:
count_df = pd.DataFrame(x_train, columns = cv.get_feature_names_out())

In [109]:
count_df.head()

,action,activist,advance,agency,ahram,ahram online,ahrar,ahrar al,ahrar al sham,aid,...,worker,world,world ahram,world ahram online,wound,wound dozen,wounded,zone,zor,zour
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [110]:
from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB()

In [111]:
from sklearn import metrics
import numpy as np
import itertools

In [112]:
classifier.fit(x_train, y_train)
pred = classifier.predict(x_test)

In [117]:
import matplotlib.pyplot as plt


In [119]:
score= metrics.accuracy_score(y_test, pred)
print("accuracy: %0.3f" % score)


accuracy: 0.730
